# GNOD Song Suggester

## Web Scraping Hot 100 from Billboard

 Scraping the current top 100 songs and their respective artists from https://www.billboard.com/charts/hot-100, and putting the information into a pandas dataframe.

In [1]:
# importing libraries

from bs4 import BeautifulSoup
import re 
import requests
import pandas as pd
from tqdm.notebook import tqdm
from random import randint

### Getting the info from the web

In [2]:
# storing link in a variable
url = "https://www.billboard.com/charts/hot-100"
# downloading  html with a get request
response = requests.get(url)
# check response status code 
response.status_code

200

In [3]:
# parsing and storing the contents of the url call
billboardsoup = BeautifulSoup(response.content, 'html.parser')
# prettifying the soup 
billboardsoup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<!--[if IE 6]>
<html id="ie6" lang="en-US">
<![endif]-->
<!--[if IE 7]>
<html id="ie7" lang="en-US">
<![endif]-->
<!--[if IE 8]>
<html id="ie8" lang="en-US">
<![endif]-->
<!--[if !(IE 6) | !(IE 7) | !(IE 8) ]><!-->
<html lang="en-US">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
<link href="https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/app/icons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<meta content="https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/app/browserconfig.xml" name="msapplication-config"/>
<meta content="https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/app/icons/icon-144x

### Querying the soup to get song title & artist

In [4]:
# getting song title

billboardsoup.select('h3.c-title.a-no-trucate')[0].get_text(strip=True)

'Easy On Me'

In [5]:
# getting artist

billboardsoup.select('span.c-label.a-font-primary-s')[0].get_text(strip=True)

'Adele'

In [6]:
# checking the amount of results

top_100 = len(billboardsoup.select('h3.c-title.a-no-trucate'))
top_100

100

In [7]:
# looping the songs

song = []
artist = []

for i in tqdm(range(top_100)):
    song.append(billboardsoup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
    artist.append(billboardsoup.select('span.c-label.a-font-primary-s')[i].get_text(strip=True))

  0%|          | 0/100 [00:00<?, ?it/s]

### Getting the dataframe and storing it

In [8]:
billboard_top100 = pd.DataFrame({'song':song,'artist':artist})

In [9]:
billboard_top100.head()

,song,artist
0,Easy On Me,Adele
1,Stay,The Kid LAROI & Justin Bieber
2,Industry Baby,Lil Nas X & Jack Harlow
3,All Too Well (Taylor's Version),Taylor Swift
4,Oh My God,Adele


In [10]:
# billboard_top100.to_csv(r'billboard_top100.csv', index = False, header = True)

## Adding more songs: Scraping top 500 songs of all times from Rolling Stone

Because no one really likes the top 100 hot songs I'm going to be using the top 500 songs of all time from Rolling Stones (the 2004 version, not the 2021) from this website https://www.cs.ubc.ca/~davet/music/list/Best9.html

### Getting the info from the web

In [11]:
# storing link in a variable
url = "https://www.cs.ubc.ca/~davet/music/list/Best9.html"
# downloading  html with a get request
response = requests.get(url)
# check response status code 
response.status_code

200

In [12]:
# parsing and storing the contents of the url call
rollingsoup = BeautifulSoup(response.content, 'html.parser')
# prettifying the soup 
rollingsoup.prettify

<bound method Tag.prettify of <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="en-us" http-equiv="Content-Language"/>
<link href="../davetmusic.css" rel="stylesheet" title="davetmusic" type="text/css"/>
<script src="../playsamples.js" type="text/javascript"></script>
<title>Rolling Stone - 500 Greatest Songs (Music Database :: Dave Tompkins)</title>
</head>
<body>
<div class="menutitle"><a href="../../index.html">Dave Tompkins</a> :: <a href="../index.html">Music Database</a></div>
<div class="menurow">
<a class="singlemenu" href="../index.html">INTRODUCTION</a>
<a class="singlemenu" href="../disc/index.html">DISCS</a>
<a class="singlemenu" href="../covers/index.html">COVERS</a>
<a class="singlemenu" href="../genre/index.html">GENRE</a>
<a class="singlemenu" href="../artisttag/index.html">ARTIST TAGS</a>
<a class="singlemenu" href="../

### Querying the soup to get song title & artist

In [13]:
# getting artist

rollingsoup.select('td:nth-child(3) > a:nth-child(2)')[0].get_text(strip=True)

'Bob Dylan'

In [14]:
# getting song

rollingsoup.select('td:nth-child(4) > a')[0].get_text(strip=True)

'Like a Rolling Stone'

In [15]:
# checking the amount of results

best500 = len(rollingsoup.select('td:nth-child(4) > a'))
best500

500

In [16]:
# looping the songs

song = []
artist = []

for i in tqdm(range(best500)):
    song.append(rollingsoup.select('td:nth-child(4) > a')[i].get_text(strip=True))
    artist.append(rollingsoup.select('td:nth-child(3) > a:nth-child(2)')[i].get_text(strip=True))

  0%|          | 0/500 [00:00<?, ?it/s]

### Getting the dataframe and storing it

In [17]:
best_500 = pd.DataFrame({'song':song,'artist':artist})

In [18]:
best_500.head()

,song,artist
0,Like a Rolling Stone,Bob Dylan
1,Satisfaction,The Rolling Stones
2,Imagine,John Lennon
3,What's Going On,Marvin Gaye
4,Respect,Aretha Franklin


In [19]:
# best_500.to_csv(r'best_500.csv', index = False, header = True)

## Creating the suggester

### The input

In [20]:
def suggester():
    song = str(input("What song do you have in mind?: ").lower().replace(" ", ''))
    # check hot 100
    check = billboard_top100[billboard_top100['song'].str.lower().str.replace(" ","").str.contains(song)]
    # get index
    index = check.index.tolist()
    if len(check) == 0:
        print("Oh well, my bad, but that song is not Hot right now. Anyway! Good for you for not following the crowds!")
    else:
        answer1 = input("Are you thinking about '" + billboard_top100.song[index].values[0] + "' by '" + billboard_top100.artist[index].values[0] + "'? [y/n]: ")
    #make a song suggestion
        if answer1.lower() == 'y':
            suggestion = billboard_top100.sample().index.tolist()
            print("Then listen to '" + billboard_top100['song'][suggestion].item() + "' by '" + billboard_top100['artist'][suggestion].item() + "' I think you'll like it.")
        else:
            print("This song is not hot, good for you for not following the crowds!")
            
suggester()

What song do you have in mind?: de
Are you thinking about 'It's The Most Wonderful Time Of The Year' by 'Andy Williams'?: y
This song is not hot, good for you for not following the crowds!


In [21]:
def coolersuggester():
    song = str(input("What song do you have in mind?: ").lower().replace(" ", ''))
    
    #get the billboard record - if available
    check = best_500[best_500['song'].str.lower().str.replace(" ","").str.contains(song)]
    #get the index of the song in the entry
    index = check.index.tolist()
    
    #run the recommendation
    if len(check) == 0:
        print("Oh well, my bad, but that song is not on the best 500 songs of all times.")
    else:
        answer1 = input("Are you thinking about '" + best_500.song[index].values[0] + "' by '" + best_500.artist[index].values[0] + "'? [y/n]: ")
    #make a song suggestion
        if answer1.lower() == 'yes':
            suggestion = best_500.sample().index.tolist()
            print("Then listen to '" + best_500['song'][suggestion].item() + "' by '" + best_500['artist'][suggestion].item() + "' I think you'll like it.")
        else:
            print("That song is not on the best 500 songs of all times. Anyway, you should listen to that suggestion!")
            
coolersuggester()

What song do you have in mind?: th
Are you thinking about 'Blowin' in the Wind' by 'Bob Dylan'?: y
That song is not on the best 500 songs of all times. Anyway, you should listen to that suggestion!


## Spotipy (API Wrappers)

In [89]:
# more libraries, you can never have enough

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint

In [90]:
client_id = str(getpass.getpass('client_id: '))
client_secret = str(getpass.getpass('client_secret: '))
# embedding getpass credentials in the access key
sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret))

client_id: ········
client_secret: ········


### Choosing some playlist to create a dataframe

In [91]:
# reference of all the playlists I'm going to use
# playlists = 
#    "the_80s_all_linedup" : ("Spotify", "37i9dQZF1DXbB7yFaZiAQX"), 
#    "all_out_80s" : ("Spotify", "37i9dQZF1DX4UtSsGT1Sbe"),
#    "aesthetic_lofi" : ("Spotify", "37i9dQZF1DXctgIf9k05Pu")
#    "modern_chill_rock" : ("Spotify", "37i9dQZF1DX2UXfvEIZvDK")
#    "soulful_blend" : ("Spotify", "37i9dQZF1DX8Md3JnnrexB")
#    "morning_coffee" : ("Spotify", "37i9dQZF1DXcgZcN2HVMoe")
#    "500_best_songs_all_times" : ("Arian", "5dxn0i8MPl6XFVVxNatd6U")
#    "best_of_the_oldies" : ("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")
#    "1001_songs_before_youdie" : ("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")
https://open.spotify.com/playlist/6aCHA5HeRfz5gJjllGZ7Of?si=b7dedd3f48924920

#    "power_ballads" : ("Spotify", "37i9dQZF1DX82Zzp6AKx64")
#    "tosing_intheshower" : ("Spotify", "37i9dQZF1DWSqmBTGDYngZ")
#    "rock_party" : ("Spotify", "37i9dQZF1DX8FwnYE6PRvL")
#    "oldies_but_goldies" : ("Spotify", "37i9dQZF1DXdmXczhgY3oW")
#    "music_workday" : ("Spotify", "37i9dQZF1DXcsT4WKI8W8r")
#    "70s_roadtrip" : ("Spotify", "37i9dQZF1DWWiDhnQ2IIru")
#    "legendary" : ("Spotify", "37i9dQZF1DWWGFQLoP9qlv")
#    "all_out_70s" : ("Spotify", "37i9dQZF1DWTJ7xPn4vNaz")
#    "all_out_90s" : ("Spotify", "37i9dQZF1DXbTxeAdrVG2l")
#    "all_out_2000s" : ("Spotify", "37i9dQZF1DX4o1oenSJRJd")
#    "all_out_2010s" : ("Spotify", "37i9dQZF1DX5Ejj0EkURtP")
#    "comfort_zone" : ("Spotify", "37i9dQZF1DWYWddJiPzbvb")
#    "mix_universal" : ("Spotify", "37i9dQZF1DWZEa9Yw06mQB")
#    "autumn_folk" : ("Spotify", "37i9dQZF1DWUOsMmqJp87m")
#    "te_para_3" : ("Spotify", "37i9dQZF1DWTSkZ0cxkrLT")
#    "mates_y_musica" : ("Spotify", "37i9dQZF1DWXTdx4nYEjeS")

### First Playlist

The 80's all lined up

In [92]:
first_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DXbB7yFaZiAQX")
first_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [93]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
first_results = get_tracks("Spotify", "37i9dQZF1DXbB7yFaZiAQX")

In [94]:
len(first_results)

150

In [95]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = first_results
for r in first_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  150 non-null    object
 1   song_id    150 non-null    object
 2   artist     150 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB


In [98]:
features_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:]))
features = pd.concat([features_info, features_info2])
features.reset_index(drop=True, inplace=True)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      150 non-null    float64
 1   energy            150 non-null    float64
 2   key               150 non-null    int64  
 3   loudness          150 non-null    float64
 4   mode              150 non-null    int64  
 5   speechiness       150 non-null    float64
 6   acousticness      150 non-null    float64
 7   instrumentalness  150 non-null    float64
 8   liveness          150 non-null    float64
 9   valence           150 non-null    float64
 10  tempo             150 non-null    float64
 11  type              150 non-null    object 
 12  id                150 non-null    object 
 13  uri               150 non-null    object 
 14  track_href        150 non-null    object 
 15  analysis_url      150 non-null    object 
 16  duration_ms       150 non-null    int64  
 1

In [99]:
first_playlist = pd.concat([playlist_info, features], axis=1)
first_playlist.reset_index()
first_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         150 non-null    object 
 1   song_id           150 non-null    object 
 2   artist            150 non-null    object 
 3   danceability      150 non-null    float64
 4   energy            150 non-null    float64
 5   key               150 non-null    int64  
 6   loudness          150 non-null    float64
 7   mode              150 non-null    int64  
 8   speechiness       150 non-null    float64
 9   acousticness      150 non-null    float64
 10  instrumentalness  150 non-null    float64
 11  liveness          150 non-null    float64
 12  valence           150 non-null    float64
 13  tempo             150 non-null    float64
 14  type              150 non-null    object 
 15  id                150 non-null    object 
 16  uri               150 non-null    object 
 1

### Second Playlist

All out 80s

In [108]:
second_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")
second_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [109]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
second_results = get_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")

In [110]:
len(second_results)

150

In [111]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = second_results
for r in second_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist2_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist2_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  150 non-null    object
 1   song_id    150 non-null    object
 2   artist     150 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB


In [112]:
features2_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features2_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:]))
features2 = pd.concat([features2_info, features2_info2])
features2.reset_index(drop=True, inplace=True)
features2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      150 non-null    float64
 1   energy            150 non-null    float64
 2   key               150 non-null    int64  
 3   loudness          150 non-null    float64
 4   mode              150 non-null    int64  
 5   speechiness       150 non-null    float64
 6   acousticness      150 non-null    float64
 7   instrumentalness  150 non-null    float64
 8   liveness          150 non-null    float64
 9   valence           150 non-null    float64
 10  tempo             150 non-null    float64
 11  type              150 non-null    object 
 12  id                150 non-null    object 
 13  uri               150 non-null    object 
 14  track_href        150 non-null    object 
 15  analysis_url      150 non-null    object 
 16  duration_ms       150 non-null    int64  
 1

In [113]:
second_playlist = pd.concat([playlist2_info, features2], axis=1)
second_playlist.reset_index()
second_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         150 non-null    object 
 1   song_id           150 non-null    object 
 2   artist            150 non-null    object 
 3   danceability      150 non-null    float64
 4   energy            150 non-null    float64
 5   key               150 non-null    int64  
 6   loudness          150 non-null    float64
 7   mode              150 non-null    int64  
 8   speechiness       150 non-null    float64
 9   acousticness      150 non-null    float64
 10  instrumentalness  150 non-null    float64
 11  liveness          150 non-null    float64
 12  valence           150 non-null    float64
 13  tempo             150 non-null    float64
 14  type              150 non-null    object 
 15  id                150 non-null    object 
 16  uri               150 non-null    object 
 1

### Third Playlist

In [114]:
third_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DXctgIf9k05Pu")
third_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [116]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
third_results = get_tracks("Spotify", "37i9dQZF1DXctgIf9k05Pu")

In [117]:
len(third_results)

50

In [119]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = third_results
for r in third_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist3_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist3_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  50 non-null     object
 1   song_id    50 non-null     object
 2   artist     50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [122]:
features3_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))
features3_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      50 non-null     float64
 1   energy            50 non-null     float64
 2   key               50 non-null     int64  
 3   loudness          50 non-null     float64
 4   mode              50 non-null     int64  
 5   speechiness       50 non-null     float64
 6   acousticness      50 non-null     float64
 7   instrumentalness  50 non-null     float64
 8   liveness          50 non-null     float64
 9   valence           50 non-null     float64
 10  tempo             50 non-null     float64
 11  type              50 non-null     object 
 12  id                50 non-null     object 
 13  uri               50 non-null     object 
 14  track_href        50 non-null     object 
 15  analysis_url      50 non-null     object 
 16  duration_ms       50 non-null     int64  
 17 

In [123]:
third_playlist = pd.concat([playlist3_info, features3_info], axis=1)
third_playlist.reset_index()
third_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         50 non-null     object 
 1   song_id           50 non-null     object 
 2   artist            50 non-null     object 
 3   danceability      50 non-null     float64
 4   energy            50 non-null     float64
 5   key               50 non-null     int64  
 6   loudness          50 non-null     float64
 7   mode              50 non-null     int64  
 8   speechiness       50 non-null     float64
 9   acousticness      50 non-null     float64
 10  instrumentalness  50 non-null     float64
 11  liveness          50 non-null     float64
 12  valence           50 non-null     float64
 13  tempo             50 non-null     float64
 14  type              50 non-null     object 
 15  id                50 non-null     object 
 16  uri               50 non-null     object 
 17 

### Fourth Playlist

In [124]:
fourth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")
fourth_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [125]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fourth_results = get_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")

In [126]:
len(fourth_results)

60

In [127]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fourth_results
for r in fourth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist4_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist4_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  60 non-null     object
 1   song_id    60 non-null     object
 2   artist     60 non-null     object
dtypes: object(3)
memory usage: 1.5+ KB


In [128]:
features4_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))
features4_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      60 non-null     float64
 1   energy            60 non-null     float64
 2   key               60 non-null     int64  
 3   loudness          60 non-null     float64
 4   mode              60 non-null     int64  
 5   speechiness       60 non-null     float64
 6   acousticness      60 non-null     float64
 7   instrumentalness  60 non-null     float64
 8   liveness          60 non-null     float64
 9   valence           60 non-null     float64
 10  tempo             60 non-null     float64
 11  type              60 non-null     object 
 12  id                60 non-null     object 
 13  uri               60 non-null     object 
 14  track_href        60 non-null     object 
 15  analysis_url      60 non-null     object 
 16  duration_ms       60 non-null     int64  
 17 

In [129]:
fourth_playlist = pd.concat([playlist4_info, features4_info], axis=1)
fourth_playlist.reset_index()
fourth_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         60 non-null     object 
 1   song_id           60 non-null     object 
 2   artist            60 non-null     object 
 3   danceability      60 non-null     float64
 4   energy            60 non-null     float64
 5   key               60 non-null     int64  
 6   loudness          60 non-null     float64
 7   mode              60 non-null     int64  
 8   speechiness       60 non-null     float64
 9   acousticness      60 non-null     float64
 10  instrumentalness  60 non-null     float64
 11  liveness          60 non-null     float64
 12  valence           60 non-null     float64
 13  tempo             60 non-null     float64
 14  type              60 non-null     object 
 15  id                60 non-null     object 
 16  uri               60 non-null     object 
 17 

### Fifth Playlist

In [130]:
fifth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")
fifth_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [131]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fifth_results = get_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")

In [132]:
len(fifth_results)

100

In [133]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fifth_results
for r in fifth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist5_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist5_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  100 non-null    object
 1   song_id    100 non-null    object
 2   artist     100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [134]:
features5_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))
features5_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      100 non-null    float64
 1   energy            100 non-null    float64
 2   key               100 non-null    int64  
 3   loudness          100 non-null    float64
 4   mode              100 non-null    int64  
 5   speechiness       100 non-null    float64
 6   acousticness      100 non-null    float64
 7   instrumentalness  100 non-null    float64
 8   liveness          100 non-null    float64
 9   valence           100 non-null    float64
 10  tempo             100 non-null    float64
 11  type              100 non-null    object 
 12  id                100 non-null    object 
 13  uri               100 non-null    object 
 14  track_href        100 non-null    object 
 15  analysis_url      100 non-null    object 
 16  duration_ms       100 non-null    int64  
 17

In [136]:
fifth_playlist = pd.concat([playlist5_info, features5_info], axis=1)
fifth_playlist.reset_index()
fifth_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         100 non-null    object 
 1   song_id           100 non-null    object 
 2   artist            100 non-null    object 
 3   danceability      100 non-null    float64
 4   energy            100 non-null    float64
 5   key               100 non-null    int64  
 6   loudness          100 non-null    float64
 7   mode              100 non-null    int64  
 8   speechiness       100 non-null    float64
 9   acousticness      100 non-null    float64
 10  instrumentalness  100 non-null    float64
 11  liveness          100 non-null    float64
 12  valence           100 non-null    float64
 13  tempo             100 non-null    float64
 14  type              100 non-null    object 
 15  id                100 non-null    object 
 16  uri               100 non-null    object 
 17

### Sixth Playlist

In [137]:
sixth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")
sixth_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [138]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
sixth_results = get_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")

In [139]:
len(sixth_results)

250

In [140]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = sixth_results
for r in sixth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist6_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist6_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  250 non-null    object
 1   song_id    250 non-null    object
 2   artist     250 non-null    object
dtypes: object(3)
memory usage: 6.0+ KB


In [143]:
features6_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features6_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features6_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:]))
features6 = pd.concat([features6_info, features6_info2, features6_info3])
features6.reset_index(drop=True, inplace=True)
features6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      250 non-null    float64
 1   energy            250 non-null    float64
 2   key               250 non-null    int64  
 3   loudness          250 non-null    float64
 4   mode              250 non-null    int64  
 5   speechiness       250 non-null    float64
 6   acousticness      250 non-null    float64
 7   instrumentalness  250 non-null    float64
 8   liveness          250 non-null    float64
 9   valence           250 non-null    float64
 10  tempo             250 non-null    float64
 11  type              250 non-null    object 
 12  id                250 non-null    object 
 13  uri               250 non-null    object 
 14  track_href        250 non-null    object 
 15  analysis_url      250 non-null    object 
 16  duration_ms       250 non-null    int64  
 1

In [144]:
sixth_playlist = pd.concat([playlist6_info, features6], axis=1)
sixth_playlist.reset_index()
sixth_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         250 non-null    object 
 1   song_id           250 non-null    object 
 2   artist            250 non-null    object 
 3   danceability      250 non-null    float64
 4   energy            250 non-null    float64
 5   key               250 non-null    int64  
 6   loudness          250 non-null    float64
 7   mode              250 non-null    int64  
 8   speechiness       250 non-null    float64
 9   acousticness      250 non-null    float64
 10  instrumentalness  250 non-null    float64
 11  liveness          250 non-null    float64
 12  valence           250 non-null    float64
 13  tempo             250 non-null    float64
 14  type              250 non-null    object 
 15  id                250 non-null    object 
 16  uri               250 non-null    object 
 1

### Seventh Playlist

In [145]:
seventh_playlist = sp.user_playlist_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")
seventh_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [146]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
seventh_results = get_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")

In [148]:
len(seventh_results)

500

In [149]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = seventh_results
for r in seventh_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist7_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist7_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  500 non-null    object
 1   song_id    500 non-null    object
 2   artist     500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [151]:
features7_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features7_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features7_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features7_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features7_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features7_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:]))
features7 = pd.concat([features7_info, features7_info2, features7_info3, features7_info4, features7_info5, features7_info6])
features7.reset_index(drop=True, inplace=True)
features7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      500 non-null    float64
 1   energy            500 non-null    float64
 2   key               500 non-null    int64  
 3   loudness          500 non-null    float64
 4   mode              500 non-null    int64  
 5   speechiness       500 non-null    float64
 6   acousticness      500 non-null    float64
 7   instrumentalness  500 non-null    float64
 8   liveness          500 non-null    float64
 9   valence           500 non-null    float64
 10  tempo             500 non-null    float64
 11  type              500 non-null    object 
 12  id                500 non-null    object 
 13  uri               500 non-null    object 
 14  track_href        500 non-null    object 
 15  analysis_url      500 non-null    object 
 16  duration_ms       500 non-null    int64  
 1

In [152]:
seventh_playlist = pd.concat([playlist7_info, features7], axis=1)
seventh_playlist.reset_index()
seventh_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         500 non-null    object 
 1   song_id           500 non-null    object 
 2   artist            500 non-null    object 
 3   danceability      500 non-null    float64
 4   energy            500 non-null    float64
 5   key               500 non-null    int64  
 6   loudness          500 non-null    float64
 7   mode              500 non-null    int64  
 8   speechiness       500 non-null    float64
 9   acousticness      500 non-null    float64
 10  instrumentalness  500 non-null    float64
 11  liveness          500 non-null    float64
 12  valence           500 non-null    float64
 13  tempo             500 non-null    float64
 14  type              500 non-null    object 
 15  id                500 non-null    object 
 16  uri               500 non-null    object 
 1

### Eighth Playlist

In [153]:
eighth_playlist = sp.user_playlist_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")
eighth_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [154]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
eighth_results = get_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")

In [155]:
len(eighth_results)

1049

In [156]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = eighth_results
for r in eighth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist8_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist8_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  1049 non-null   object
 1   song_id    1049 non-null   object
 2   artist     1049 non-null   object
dtypes: object(3)
memory usage: 24.7+ KB


In [157]:
features8_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features8_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features8_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features8_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features8_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features8_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features8_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features8_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features8_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features8_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features8_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features8 = pd.concat([features8_info, features8_info2, features8_info3, features8_info4, features8_info5, features8_info6, features8_info7, features8_info8, features8_info9, features8_info10, features8_info11])
features8.reset_index(drop=True, inplace=True)
features8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1049 non-null   float64
 1   energy            1049 non-null   float64
 2   key               1049 non-null   int64  
 3   loudness          1049 non-null   float64
 4   mode              1049 non-null   int64  
 5   speechiness       1049 non-null   float64
 6   acousticness      1049 non-null   float64
 7   instrumentalness  1049 non-null   float64
 8   liveness          1049 non-null   float64
 9   valence           1049 non-null   float64
 10  tempo             1049 non-null   float64
 11  type              1049 non-null   object 
 12  id                1049 non-null   object 
 13  uri               1049 non-null   object 
 14  track_href        1049 non-null   object 
 15  analysis_url      1049 non-null   object 
 16  duration_ms       1049 non-null   int64  


In [158]:
eighth_playlist = pd.concat([playlist8_info, features8], axis=1)
eighth_playlist.reset_index()
eighth_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         1049 non-null   object 
 1   song_id           1049 non-null   object 
 2   artist            1049 non-null   object 
 3   danceability      1049 non-null   float64
 4   energy            1049 non-null   float64
 5   key               1049 non-null   int64  
 6   loudness          1049 non-null   float64
 7   mode              1049 non-null   int64  
 8   speechiness       1049 non-null   float64
 9   acousticness      1049 non-null   float64
 10  instrumentalness  1049 non-null   float64
 11  liveness          1049 non-null   float64
 12  valence           1049 non-null   float64
 13  tempo             1049 non-null   float64
 14  type              1049 non-null   object 
 15  id                1049 non-null   object 
 16  uri               1049 non-null   object 


### Ninth Playlist

In [159]:
ninth_playlist = sp.user_playlist_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")
ninth_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [160]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
ninth_results = get_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")

In [161]:
len(ninth_results)

1032

In [162]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = ninth_results
for r in ninth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist9_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})
playlist9_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_name  1032 non-null   object
 1   song_id    1032 non-null   object
 2   artist     1032 non-null   object
dtypes: object(3)
memory usage: 24.3+ KB


In [163]:
features9_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features9_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features9_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features9_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features9_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features9_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features9_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features9_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features9_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features9_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features9_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features9 = pd.concat([features9_info, features9_info2, features9_info3, features9_info4, features9_info5, features9_info6, features9_info7, features9_info8, features9_info9, features9_info10, features9_info11])
features9.reset_index(drop=True, inplace=True)
features9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1032 non-null   float64
 1   energy            1032 non-null   float64
 2   key               1032 non-null   int64  
 3   loudness          1032 non-null   float64
 4   mode              1032 non-null   int64  
 5   speechiness       1032 non-null   float64
 6   acousticness      1032 non-null   float64
 7   instrumentalness  1032 non-null   float64
 8   liveness          1032 non-null   float64
 9   valence           1032 non-null   float64
 10  tempo             1032 non-null   float64
 11  type              1032 non-null   object 
 12  id                1032 non-null   object 
 13  uri               1032 non-null   object 
 14  track_href        1032 non-null   object 
 15  analysis_url      1032 non-null   object 
 16  duration_ms       1032 non-null   int64  


In [164]:
ninth_playlist = pd.concat([playlist9_info, features9], axis=1)
ninth_playlist.reset_index()
ninth_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         1032 non-null   object 
 1   song_id           1032 non-null   object 
 2   artist            1032 non-null   object 
 3   danceability      1032 non-null   float64
 4   energy            1032 non-null   float64
 5   key               1032 non-null   int64  
 6   loudness          1032 non-null   float64
 7   mode              1032 non-null   int64  
 8   speechiness       1032 non-null   float64
 9   acousticness      1032 non-null   float64
 10  instrumentalness  1032 non-null   float64
 11  liveness          1032 non-null   float64
 12  valence           1032 non-null   float64
 13  tempo             1032 non-null   float64
 14  type              1032 non-null   object 
 15  id                1032 non-null   object 
 16  uri               1032 non-null   object 


In [178]:
huge_playlist = pd.concat([first_playlist, second_playlist, third_playlist, fourth_playlist, 
                           fifth_playlist, sixth_playlist, seventh_playlist, eighth_playlist, 
                           ninth_playlist])
huge_playlist.reset_index(drop=True, inplace=True)
huge_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3341 entries, 0 to 3340
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         3341 non-null   object 
 1   song_id           3341 non-null   object 
 2   artist            3341 non-null   object 
 3   danceability      3291 non-null   float64
 4   energy            3291 non-null   float64
 5   key               3291 non-null   float64
 6   loudness          3291 non-null   float64
 7   mode              3291 non-null   float64
 8   speechiness       3291 non-null   float64
 9   acousticness      3291 non-null   float64
 10  instrumentalness  3291 non-null   float64
 11  liveness          3291 non-null   float64
 12  valence           3291 non-null   float64
 13  tempo             3291 non-null   float64
 14  type              3291 non-null   object 
 15  id                3291 non-null   object 
 16  uri               3291 non-null   object 


In [172]:
# count duplicated values
huge_playlist.duplicated(subset='song_name', keep='first').sum()

485

In [179]:
huge_playlist = huge_playlist.drop_duplicates()
huge_playlist.reset_index(drop=True, inplace=True)
huge_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3207 entries, 0 to 3206
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         3207 non-null   object 
 1   song_id           3207 non-null   object 
 2   artist            3207 non-null   object 
 3   danceability      3157 non-null   float64
 4   energy            3157 non-null   float64
 5   key               3157 non-null   float64
 6   loudness          3157 non-null   float64
 7   mode              3157 non-null   float64
 8   speechiness       3157 non-null   float64
 9   acousticness      3157 non-null   float64
 10  instrumentalness  3157 non-null   float64
 11  liveness          3157 non-null   float64
 12  valence           3157 non-null   float64
 13  tempo             3157 non-null   float64
 14  type              3157 non-null   object 
 15  id                3157 non-null   object 
 16  uri               3157 non-null   object 
